# Load bigfish detection results


In [1]:
import os
import napari
import numpy as np
import pandas as pd
import trackpy as tp
import matplotlib.pyplot as plt

import bigfish
import bigfish.plot as plot
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack

from copy import deepcopy
from getTranscriptionSites import *
from dask.array.image import imread as imr
from bigfish.detection.utils import get_object_radius_pixel
from buildReferenceSpot import buildReferenceSpotFromImages
from runBigfishDetection import getSpotAndClusters, saveSpotsNPZ,reorderZstack
pd.set_option('display.max_rows', 1500)

/tmp/ipykernel_5241/809215763.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import session_info
session_info.show()

In [3]:
print(np.__file__)

/home/raphael/miniconda3/envs/bigfishlive/lib/python3.9/site-packages/numpy/__init__.py


## Specify Input and Output folders, identifiers

In [4]:
homeFolder = '/media/raphael/data/works/full_data_movie/'
nameKey = 'c3_21_c3_45_ON_1_w1SPI-488_s'

imsQ = '3'
cellNumber = '25'

nucleiStackForm = nameKey+imsQ+"_cell_"

pathToTimeSequenceFullView = homeFolder+nameKey+imsQ+'/*.tif'
cell_Crop_Folder = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/'
pathToTimeSequenceCell = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/*.tif'
moviePath = homeFolder+nameKey+imsQ

sequenceCell = imr(pathToTimeSequenceCell)
mipSequenceCell = np.max(sequenceCell, axis=1)
MaxTimePoint = sequenceCell.shape[0] -1

## Load detection and Clustering Results

In [5]:
spcl = np.load(cell_Crop_Folder+str(cellNumber)+'_spots_and_clusters.npz',allow_pickle=True)

spotsFrame = spcl['spotsFrame']
clustersFrames = spcl['clustersFrames']
ThresholdFrames = spcl['ThresholdFrames']
reference_spot = spcl['reference_spot']
refSpot = deepcopy(reference_spot)

## Get transcription sites using high confidence track information

In [6]:
particle_1 = pd.read_pickle(moviePath+'/cellNumber_'+str(cellNumber)+'_particle_1.pkl') 
particle_1.index = np.arange(0,len(particle_1))
particle_1.track_length = len(particle_1)

txs1 = getTranscrtiptionSites(mipSequenceCell, particle_1, MaxTimePoint, spotsFrame, clustersFrames, 0, 1100)

particle_2 = pd.read_pickle(moviePath+'/cellNumber_'+str(cellNumber)+'_particle_2.pkl') 
particle_2.index = np.arange(0,len(particle_2))
particle_2.track_length = len(particle_2)

txs2 = getTranscrtiptionSites(mipSequenceCell, particle_2, MaxTimePoint, spotsFrame, clustersFrames, 1, 1100)

In [7]:
## Convert Tx information to napari visualisable format

newClusterFrame = []
for hh in range(len(txs1)):
    if np.sum(txs1.iloc[hh,1:-1])==0:
        newClusterFrame.append(np.array([], dtype=np.int64).reshape((0,5)))
    else:
        newClusterFrame.append(np.array(txs1.iloc[hh,1:]).reshape((1,5)).astype(np.int64))
        
for hh in range(len(txs2)):
    if np.sum(txs2.iloc[hh,1:-1])==0:
        newClusterFrame[hh] = np.vstack([newClusterFrame[hh],np.array([], dtype=np.int64).reshape((0,5))])
    else:
        newClusterFrame[hh] = np.vstack([newClusterFrame[hh], np.array(txs2.iloc[hh,1:]).reshape((1,5)).astype(np.int64)])


## Visualise results

In [8]:
def getDetectedPointsForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-1])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:]
    return pts_coords

def getDetectedClustersForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-3])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:3]
    return pts_coords

def set_pts_features(pts_layer, cls_layer, pts_coordinates, cluster_coordinate, step): #TxLayer
    # step is a 4D coordinate with the current slider position for each dim
    frameNumber = step[0]  # grab the leading ("time") coordinate
    pts_layer.data = getDetectedPointsForFrame(pts_coordinates,frameNumber)
    cls_layer.data = getDetectedClustersForFrame(cluster_coordinate,frameNumber)

pts_coordinates = spotsFrame
cluster_coordinate = newClusterFrame#clustersFrames
viewer = napari.Viewer()
image_layer = viewer.add_image(
        mipSequenceCell, colormap='green' #maxImageCell
        )
if image_layer.data.ndim == 4:
    bigfishSpots = spotsFrame
elif image_layer.data.ndim == 3:
    bigfishSpots = getDetectedPointsForFrame(pts_coordinates,int(np.shape(mipSequenceCell)[0]/2))
    
bigfish_Spots = viewer.add_points(
        getDetectedPointsForFrame(pts_coordinates,int((np.shape(mipSequenceCell)[0]-1)/2)),
        face_color='#00000000',
        size=4,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='white',
        face_color_cycle = ['white'],
        name = 'bigFish Detected Spots'
        )

bigfish_clusters = viewer.add_points(
        getDetectedClustersForFrame(cluster_coordinate,int((np.shape(mipSequenceCell)[0]-1)/2)),
        face_color='#00000000',
        size=8,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='red',
        face_color_cycle = ['red'],
        symbol='diamond',
        name = 'bigFish Clusters'
        )

viewer.dims.events.current_step.connect(
        lambda event: set_pts_features(bigfish_Spots, bigfish_clusters, pts_coordinates, cluster_coordinate, event.value)
        )

<function __main__.<lambda>(event)>

In [9]:
type(cluster_coordinate[0])

numpy.ndarray

In [19]:
txs2

,frame,z,y,x,mrna,cluster_id
0,0,2.0,36.0,59.0,5.0,1
1,1,25.0,51.0,79.0,5.0,1
2,2,19.0,47.0,81.0,7.0,1
3,3,20.0,47.0,69.0,6.0,1
4,4,9.0,46.0,55.0,4.0,1
5,5,21.0,41.0,66.0,7.0,1
6,6,12.0,38.0,43.0,6.0,1
7,7,26.0,40.0,48.0,8.0,1
8,8,7.0,34.0,56.0,7.0,1
9,9,14.0,38.0,54.0,7.0,1


## Save transcription site track information 

In [10]:
txs1.to_csv(moviePath+'/cellNumber_'+str(cellNumber)+'_particle_1_tracks.csv')
txs2.to_csv(moviePath+'/cellNumber_'+str(cellNumber)+'_particle_2_tracks.csv')

trackData = np.savez(moviePath+'/cellNumber_'+str(cellNumber)+'trackData',
                    tx1 = txs1,
                    tx2 = txs2,
                    newClusterFrame = newClusterFrame,
                    allow_pickle=True)

## Save spots 

In [11]:
import pandas as pd 
import xlsxwriter

def save_xls_sheets(pts_coordinates, savepath, name):
    
    if not os.path.exists(savepath):
        os.makedirs(savepath)
    
    with pd.ExcelWriter(savepath + name + '.xlsx', engine ='xlsxwriter') as writer :
        for i in range(len(pts_coordinates)):
            sheet = pd.DataFrame(data = pts_coordinates[i], columns=['index','axis-0','axis-1'])
            sheet.to_excel(writer,sheet_name='Feuille ' + str(i), index = False)
            
        print('Done')

In [12]:
name = 'c3_21_c3_45_ON_1_w1SPI-488_s3' + '_cell_'+ str(cellNumber)
savepath = moviePath + '/' + 'spotsFrame/' 

save_xls_sheets(pts_coordinates,savepath,name)

Done


In [13]:
moviePath

'/media/raphael/data/works/full_data_movie/c3_21_c3_45_ON_1_w1SPI-488_s3'